In [5]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
#!pip install geopy
from geopy.distance import geodesic
import math
import multiprocessing as mp
from scipy.spatial import cKDTree


In [6]:
cards_df = pd.read_csv('sd254_cards.csv')  # Replace 'file_name.csv' with your CSV file path
users_df = pd.read_csv('sd254_users.csv') 
#trans_df = pd.read_csv('User0_credit_card_transactions.csv') 
cards_df = pd.DataFrame(cards_df)
users_df = pd.DataFrame(users_df)
cards_df.drop(columns=['Card Brand'], inplace=True)
cards_df.columns
users_df.reset_index()
users_df.columns
df_merged = users_df.merge(cards_df, left_index=True, right_on='User')
df_merged


,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,...,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,...,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,...,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,...,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,...,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,Annika Russell,47,67,1973,1,Female,276 Fifth Boulevard,NaN,Elizabeth,NJ,...,Credit,300609782832003,01/2024,663,YES,1,$6900,11/2000,2013,No
6142,Annika Russell,47,67,1973,1,Female,276 Fifth Boulevard,NaN,Elizabeth,NJ,...,Credit,4718517475996018,01/2021,492,YES,2,$5700,04/2012,2012,No
6143,Juelz Roman,66,60,1954,2,Male,259 Valley Boulevard,NaN,Camp Hill,PA,...,Credit,5929512204765914,08/2020,237,NO,2,$9200,02/2012,2012,No
6144,Kenia Harris,21,60,1998,11,Female,472 Ocean View Street,NaN,Merrimack,NH,...,Debit,5589768928167462,01/2020,630,YES,1,$28074,01/2020,2020,No


In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from geopy.distance import geodesic
from scipy.spatial import cKDTree

# Function to generate a random date within the last 30 days
def random_date_within_last_30_days():
    today = datetime.today()
    random_date = today - timedelta(days=np.random.randint(0, 31))
    return random_date.date()

# Function to precompute nearby cities
def precompute_nearby_cities(unique_cities, radius_km=1000):
    coords = unique_cities[['Latitude', 'Longitude']].to_numpy()
    tree = cKDTree(coords)
    nearby_cities_dict = {}

    for i, row in unique_cities.iterrows():
        # Find indices of nearby points
        nearby_indices = tree.query_ball_point([row['Latitude'], row['Longitude']], radius_km / 6371.0)  # Radius in radians
        nearby_cities = unique_cities.iloc[nearby_indices]
        nearby_cities_dict[(row['Latitude'], row['Longitude'])] = nearby_cities

    return nearby_cities_dict

# Function to get the first nearby city
def get_first_nearby_city(lat, lon, nearby_cities_dict):
    key = (lat, lon)
    if key in nearby_cities_dict:
        nearby_cities = nearby_cities_dict[key]
        if not nearby_cities.empty:
            return nearby_cities.sample(1).iloc[0]  # Randomly pick one nearby city
    return None

# Parameters
same_city_prob = 0.7
nearby_city_prob = 0.29
random_city_prob = 0.01
time_day_prob = 0.6
amount_probs = [0.6, 0.3, 0.1]
error_probs = [0.9, 0.05, 0.05]
fraud_probs = [0.80, 0.20]

# Function to generate transactions
def generate_transactions(card, num_transactions, nearby_cities_dict):
    transactions = []
    for _ in range(num_transactions):
        city_choice = np.random.choice(['same', 'nearby', 'random'], p=[same_city_prob, nearby_city_prob, random_city_prob])

        if city_choice == 'same':
            city, state, lat, lon = card['City'], card['State'], card['Latitude'], card['Longitude']

        elif city_choice == 'nearby':
            nearby_city = get_first_nearby_city(card['Latitude'], card['Longitude'], nearby_cities_dict)
            if nearby_city is not None:
                city, state, lat, lon = nearby_city['City'], nearby_city['State'], nearby_city['Latitude'], nearby_city['Longitude']
            else:
                city, state, lat, lon = card['City'], card['State'], card['Latitude'], card['Longitude']

        else:  # Random
            random_row = df_merged.sample(1).iloc[0]
            city, state, lat, lon = random_row['City'], random_row['State'], random_row['Latitude'], random_row['Longitude']

        # Date logic
        date = random_date_within_last_30_days()

        # Time logic
        time_frame = np.random.choice(['day', 'night'], p=[time_day_prob, 1 - time_day_prob])
        if time_frame == 'day':
            time = f"{np.random.randint(6, 20):02}:{np.random.randint(0, 60):02}"
        else:
            time = f"{np.random.choice([np.random.randint(0, 6), np.random.randint(20, 24)]):02}:{np.random.randint(0, 60):02}"

        # Amount logic
        amount_category = np.random.choice([1, 2, 3], p=amount_probs)
        if amount_category == 1:
            amount = round(np.random.uniform(1, 100), 2)
        elif amount_category == 2:
            amount = round(np.random.uniform(101, 500), 2)
        else:
            amount = round(np.random.uniform(500, 2000), 2)

        # Error logic
        error = np.random.choice(['None', 'Insufficient Funds', 'Card Declined'], p=error_probs)

        # Fraud logic
        is_fraud = 0
        if city_choice == 'random':
            is_fraud = 1
        elif city_choice == 'nearby':
            is_fraud = np.random.choice([0, 1])
        elif amount_category == 3:
            is_fraud = np.random.choice([1, 0], p=[0.9, 0.1])
        elif time_frame == 'night' and error != 'None':
            is_fraud = np.random.choice([0, 1], p=[0.8, 0.2])

        # Append transaction
        transactions.append({
            'Date': date,
            'Time': time,
            'Amount': amount,
            'Errors?': error,
            'Use Chip': np.random.choice([0, 1]),
            'Is Fraud?': is_fraud,
            'Card Number': card['Card Number'],
            'CVV': card['CVV'],
            'Expires': card['Expires'],
            'Has Chip': np.random.choice([0, 1]),
            'City': city,
            'State': state,
            'Latitude': lat,
            'Longitude': lon,
        })
    return transactions

# Precompute nearby cities
unique_cities = df_merged[['City', 'State', 'Latitude', 'Longitude']].drop_duplicates()
nearby_cities_dict = precompute_nearby_cities(unique_cities)

# Generate dataset
all_transactions = []
for _, card in df_merged.iterrows():
    all_transactions.extend(generate_transactions(card, np.random.randint(20, 31), nearby_cities_dict))

# Convert to DataFrame
transactions_df = pd.DataFrame(all_transactions)
transactions_df['Card Number'] = transactions_df['Card Number'].astype(str)
transactions_df['CVV'] = transactions_df['CVV'].astype(str)

# Save to CSV or use as needed
transactions_df.to_csv('mock_transactions.csv', index=False)
print(transactions_df.head())


         Date   Time  Amount             Errors?  Use Chip  Is Fraud?  \
0  2024-11-29  05:19  190.54                None         1          0   
1  2024-11-06  23:38   69.87  Insufficient Funds         0          0   
2  2024-11-27  14:56   52.39       Card Declined         0          0   
3  2024-11-07  11:28  189.13                None         0          0   
4  2024-11-09  15:45   19.62                None         0          1   

        Card Number  CVV  Expires  Has Chip      City State  Latitude  \
0  4344676511950444  623  12/2022         1  La Verne    CA     34.15   
1  4344676511950444  623  12/2022         0  La Verne    CA     34.15   
2  4344676511950444  623  12/2022         0  La Verne    CA     34.15   
3  4344676511950444  623  12/2022         0    Pomona    CA     34.06   
4  4344676511950444  623  12/2022         0    Pomona    CA     34.06   

   Longitude  
0    -117.76  
1    -117.76  
2    -117.76  
3    -117.79  
4    -117.79  


In [8]:
# transactions_df.to_csv('mock_transactions.csv', index=False)
# print(transactions_df.head())